# Projeto 2 - Ciência dos Dados

**Nome: Thiago Maitan Pegorer**

**Nome: Raphael Jacob Butori**

## Classificador automático de sentimento

### Preparando o ambiente no jupyter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [3]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle


## Autenticando no Twitter

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @pegorer_thi

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

# Projeto

### Escolha de um produto e coleta das mensagens

In [5]:
#Produto escolhido:
produto = 'Samsung'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

In [6]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
            break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)


In [7]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

In [8]:
tw=pd.read_excel("Samsung.xlsx")
tw.head()

,Treinamento,Classificador
0,alerta de oferta: samsung galaxy a30 a partir ...,0
1,apesar de amar a samsung. vi hoje que a #xiaom...,0
2,odeio samsung. pqppp,1
3,@hearthstoneness @cissamagazine na loja tbm né...,0
4,tenho que comprar um celular novo mas 0 vontad...,1


### Limpando pontuação do excel

In [9]:
import string
#string punctuation contem todas as pontuacoes ja
pontuacao = string.punctuation

ponto = ["\n", "\t"]

#criando uma lista
limpeza = []

for tweet in tw["Treinamento"]:
    x = ''
    for palavra in tweet:
        if palavra in ponto:
            x += " "
        elif palavra not in pontuacao:
            x += palavra
    limpeza.append(x)

tw_sam =  pd.DataFrame()
tw_sam['Treinamento'] = limpeza
tw_sam['Classificador'] = tw['Classificador']

tw_sam.head()



,Treinamento,Classificador
0,alerta de oferta samsung galaxy a30 a partir d...,0
1,apesar de amar a samsung vi hoje que a xiaomi ...,0
2,odeio samsung pqppp,1
3,hearthstoneness cissamagazine na loja tbm né n...,0
4,tenho que comprar um celular novo mas 0 vontad...,1


In [10]:
relevante=tw_sam.loc[tw_sam.Classificador==1]
#print(len(relevante))
irrelevante=tw_sam.loc[tw_sam.Classificador==0]
#print(len(irrelevante))
j=relevante.Treinamento
k=irrelevante.Treinamento

### Elaboração das funções 

In [11]:
#anexando os tweets em uma lista
def split(relevanciaT):
    listapalavras=[]
    for e in relevanciaT:
        tweet=e.split()
        for palavra in tweet:
            listapalavras.append(palavra)
    return listapalavras
#print(split(j))



In [12]:
#frequencia palavras
def FP(tweetsplit):
    y={}
    for x in tweetsplit:
        if x not in y:
            y[x]=1
        if x in y:
            y[x]+=1
    return y
#print(FP(listarelev))

In [13]:
def FRTW(y,relevancia):
    frelattw={}
    for x,z in y.items():
        b=z/len(relevancia)
        frelattw[x]=b
    return frelattw
#print(FRTW(y))

#frequencia relativa por tweet

In [14]:
#calcula a frequencia relativa das palavras com relacao ao todo das palavras
def FRP(y,lpal):
    frelatpal={}
    for x,z in y.items():
        b=z/len(lpal)
        frelatpal[x]=b
    return frelatpal
#print(FRP(y))



#### Criação das variáveis

* Relevante

In [15]:
Rsplitado=split(j)
Rcontagem=FP(Rsplitado)
Rfrtw=FRTW(Rcontagem,relevante)#frequencia relativa das palavras em tweets relevantes
Rfrp=FRP(Rcontagem,Rsplitado)

* Irrelevante 

In [19]:
Isplitado=split(k)
Icontagem=FP(Isplitado)
Ifrtw=FRTW(Icontagem,irrelevante)#frequencia relativa das palavras em tweets irrelevantes
Ifrp=FRP(Icontagem,Isplitado)

### Classificação FRTW (Frequencia Relativa à Tweets)

In [16]:
tweet='portal lucena informa: celular dobrável da samsung chega às lojas https://t.co/qzduwjrvdk'

In [20]:
def ClassificadorFRTW(tweet):
    pontuacao = string.punctuation

    ponto = ["\n", "\t"]

    x = ''
    for palavra in tweet:
        if palavra in ponto:
            x += " "
        elif palavra not in pontuacao:
            x += palavra
    twsplit=x.split()
    total=0
    irrelevancia=0
    relevancia=0
    resultado=0
    for x in twsplit:
        if x !='samsung':
            if x in Rfrtw:
                relevancia+=Rfrtw[x]
            print(x)
            print ('relevancia:{0}'.format(relevancia))
    for x in twsplit:
        if x !='samsung':
            if x in Ifrtw:
                irrelevancia+=Ifrtw[x]
            print(x)
            print ('irrelevancia:{0}'.format(irrelevancia))
    total=(relevancia)-(irrelevancia)
    print('total:{0}'.format(total))
    if total>0:
        resultado='relevante'
    else:
        resultado='irrelevante'
    return resultado

In [21]:
ClassificadorFRTW(tweet)

portal
relevancia:0.016
lucena
relevancia:0.032
informa
relevancia:0.048
celular
relevancia:0.304
dobrável
relevancia:0.33599999999999997
da
relevancia:0.824
chega
relevancia:0.84
às
relevancia:0.856
lojas
relevancia:0.872
httpstcoqzduwjrvdk
relevancia:0.872
portal
irrelevancia:0.011428571428571429
lucena
irrelevancia:0.022857142857142857
informa
irrelevancia:0.03428571428571429
celular
irrelevancia:0.14285714285714285
dobrável
irrelevancia:0.16
da
irrelevancia:0.4742857142857143
chega
irrelevancia:0.48571428571428577
às
irrelevancia:0.4971428571428572
lojas
irrelevancia:0.5085714285714287
httpstcoqzduwjrvdk
irrelevancia:0.5085714285714287
total:0.3634285714285713


'relevante'

Apesar de a principio aparentar ser mais viável utilizar como referência para o classificador a frequência por tweet da palavra ao invés de sua frequência relativa com relação ao todo, nos deparamos com problemas com relação à influência que artigos estavam tendo no resultado final , uma vez que os mesmos estavam presentes na maioria dos relevantes, assim o impacto que os mesmos tinham no tweet ser relevante ou não acabava sendo imenso, com isso partimos para a opção seguinte, o FRP.

### Classificador FRP (Frequencia Relativa à Palavras)

In [25]:
tweet='amo ter um iphone'

In [26]:
def ClassificadorFRP(tweet):
    pontuacao = string.punctuation

    ponto = ["\n", "\t"]

    x = ''
    for palavra in tweet:
        if palavra in ponto:
            x += " "
        elif palavra not in pontuacao:
            x += palavra
    twsplit=x.split()
    total=0
    irrelevancia=0
    relevancia=0
    resultado=0
    for x in twsplit:
        if x !='samsung':
            if x in Rfrp:
                relevancia+=Rfrp[x]
            print(x)
            print ('relevancia:{0}'.format(relevancia))
    for x in twsplit:
        if x !='samsung':
            if x in Ifrp:
                irrelevancia+=Ifrp[x]
            print(x)
            print ('irrelevancia:{0}'.format(irrelevancia))
    total=(relevancia)-(irrelevancia)
    print('total:{0}'.format(total))
    if total>0:
        resultado='relevante'
    else:
        resultado='irrelevante'
    return resultado
        

In [27]:
ClassificadorFRP(tweet)

amo
relevancia:0.0013215859030837004
ter
relevancia:0.0052863436123348016
um
relevancia:0.02026431718061674
iphone
relevancia:0.025110132158590308
amo
irrelevancia:0.0013550135501355014
ter
irrelevancia:0.0027100271002710027
um
irrelevancia:0.0220189701897019
iphone
irrelevancia:0.028794037940379408
total:-0.0036839057817891


'irrelevante'

## Teste

# Progresso possível

Além da óbvia melhoria de precisão aumentando o grupo de controle feito manualmente,e a divisão em subcategorias sobre o tipo de feedback recebido, não apenas se consiste em um feedback de usuário ou não(o que foi considerado como relevante neste projeto), uma possível melhoria seria a eliminação de artigos como grande influência no classificador.Tratando de um grupo de controle pequeno quando comparado à escala empresarial, a presença de artigos pessoais em um texto faz com que este tenda fortemente para o lado relevante, uma vez que os textos considerados relevantes vem de usuários expressando o feedback, um problema surge quando companhias fazem anúncios com o intúito de aproximar o usuário, utilizando artigos informais e confundindo o classificador, com uma base de dados maior como treinamento este problema caminharia para uma solução automática uma vez que estes anúncios seriam levados em conta, porém tratando de uma escala menor talvez fosse interessante abanar a influência deles sob o programa, isso seria feito criando uma lista com os artigos mais frequentemente utilizados em anúncios (que poderia ser criada cruzando uma lista de artigos com um programa que examina a frequencia dos mesmos em um DF de anuncios) ou por inserção manual de uma parcela maior de anúncios desse caráter no dataframe inicial.


# Por que não usar o Treinamento para validação?

A utilização do grupo de treinamento também como grupo de validação não corresponde à como o classificador atuaria numa situação com um novo tweet , utilizando-o no grupo de treinamento todas as palavras da frase teriam um valor atribuído a elas , uma situação utópica, e o resultado esperado deveria ser uma precisão também muito acima do que como o classificador realmente atuaria em ação.


# Naive bayes

Em qualquer situação onde é possivel atribuir uma condição de sucesso ou não sucesso à uma variável , não necessáriamente em um padrão visível e não necessáriamente apenas uma variável , é possivel aplicar o naive bayes em uma série de variáveis atreladas à um resultado , a probabilidade de sucesso atrelada à cada variável disposta em uma situação a ser prevista resultará em uma soma que permite fazer uma previsão baseada em comportamento prévio. como exemplo pode se usar qualquer situação: -> aninha foi x dia ao parque , estava sol e ela estava de chinelo -> aninha não foi ao parque no dia z , estava chuva e ela estava de chinelo (...) e assim vai no final conseguindo atribuir valores de probabilidade de sucesso á cada uma das variáveis em questão
